# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

Import Statements:



In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

API Key Setup:



Context Setup:
- Creates a list to store conversation context
- Adds the user's message as a system message


API Call:

- Makes a call to OpenAI's API
- Uses GPT-3.5-Turbo model
- temperature=0 means responses will be more deterministic/focused
- Returns the response

Response Handling:
- Extracts and returns the actual response content from the API
- response: This is the complete response object returned from the OpenAI API call
- response.choices: The API returns an array of choices (possible responses)
In most cases, we only get one response back, which is why we access index [0]
- choices[0].message: Each choice contains a message object
The message object contains the actual response from the model
- .content: This extracts the actual text content from the message
This is the final, clean response text we want to use

context =[  ] -> Creates an empty list to store the conversation context
 - Adds a message with two key components:
  - **role: "system" indicates this is a system-level instruction**
  - **content: Contains the actual user message**


This format is required by OpenAI's chat API

API Call Setup:
- Temperature from '0 to 1' (we dont want it to be **creative** since these are SQL responses so 0 is good for now)


In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = [] #The context is like preparing a question

    context.append({'role':'system', "content": user_message})
    #The API call is like asking the question

    response = client.chat.completions.create(    #The API call is like asking the question

            model="gpt-3.5-turbo", # Specifies which GPT model to use
            messages=context, # Passes our formatted context
            temperature=0, # Controls randomness (0 = most deterministic)
        )    
    return (response.choices[0].message.content)    

#This RETURN line is accessing the response from GPT-3.5-Turbo in a specific way. 
#Think of it like opening a nested package:
  #response - This is the entire response object from OpenAI's API call
  #.choices - The API returns an array of possible responses (choices)
  #[0] - We're accessing the first (and usually only) response in the array
  #.message - Each choice contains a message object
  #.content - This gets the actual text content of the message

In [3]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample / # ENTER A TABLE COLUMNS HERE,  # ENTER A TABLE DEFINITATIONS HERE
data = {
    'table': [],  # List of table columns
    'definition': []  # List of corresponding definitions
}

# Create DataFrame
df = pd.DataFrame(data)
print(df)

Empty DataFrame
Columns: [table, definition]
Index: []


In [4]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

#'\n' creates a new line
#here, it iterates through each table and definition row and evaluates each expresion and combine 
#the tables into a readable format

print(text_tables)

In [5]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Question:
{question}
"""


In [6]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="Which movie won the most Oscars")

  - format() Method: This line is filling the placeholders in the prompt_question_tables template you defined earlier.
  - The {tables} placeholder is being replaced by the content stored in text_tables (the result of the earlier code which generated table and definition pairs).
  - The {question} placeholder is being replaced by the question "Which movie won the most Oscars".
**Purpose: This dynamically generates a prompt for GPT-3.5 to help identify which tables from your database structure are necessary to answer the question.**

In [7]:
print(return_OAI(pqt1))

```json
{
    "tables": {
        "movies": "Contains information about movies",
        "awards": "Contains information about awards won by movies"
    }
}
```


In [8]:
pqt3 = prompt_question_tables.format(tables=text_tables,question="Which actors have won the most Oscars?")

In [9]:
print(return_OAI(pqt3))

```json
{
    "tables": ["actors", "awards"]
}
```


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative


In [11]:
# 1. Define Tables and Corresponding Descriptions
data = {
    'table': ['Gross Revenue', 'Movie', 'Actors','Director', 'Award', 'Year of Release'],
    'definition': [
        'Contains customer details such as how much money the studio earned with its premiere',
        'Contains the name of the movie',
        'Contains the name of the actors',
        'Contains the name of the director',
        'Contains the number of awards and specifies the Category of the award',
        'Contains the year of release.'
    ]
}

# Create DataFrame
df = pd.DataFrame(data)


In [14]:
pqt1 = prompt_question_tables.format(tables=text_tables, question="Which movie won the most Oscars?")
print(return_OAI(pqt1))

# Prompt 2 (you can complete this one)
pqt2 = prompt_question_tables.format(tables=text_tables, question="Which actors have won the most Oscars?")
print(return_OAI(pqt2))

# Prompt 3 (Feel free to experiment and complete this one)
pqt3 = prompt_question_tables.format(tables=text_tables, question="Which directors have won the most Oscars?")
print(return_OAI(pqt3))

# Prompt 4 (Be creative and complete this one)
pqt4 = prompt_question_tables.format(tables=text_tables, question="Which movie had the most gross revenue?") 
print(return_OAI(pqt4))

```json
{
    "tables": ["movies", "awards"]
}
```
```json
{
    "tables": ["actors", "awards"]
}
```
```json
{
    "tables": ["directors", "awards"]
}
```
```json
{
    "tables": ["movies"]
}
```


 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In this lab, we used GPT-3.5 to generate SQL queries based on table names and definitions, leveraging prompts that ask GPT-3.5 to identify which database tables are relevant to answer specific questions. The goal was to see how well GPT-3.5 could interpret table structures and provide accurate table mappings for SQL queries.

Prompts Tested:
I tested a series of prompts to evaluate how GPT-3.5 would respond in different scenarios, focusing on querying information related to movies, actors, and awards.

Examples of Prompts Tested:

"Which movie won the most Oscars?"
Result: GPT-3.5 identified the relevant tables as "movies" and "awards".
"Which actors have won the most Oscars?"
Result: GPT-3.5 returned the tables "actors" and "awards", showing consistency in correctly mapping awards-related queries.
"Which directors have won the most Oscars?"
Result: The model correctly mapped to "directors" and "awards" tables.
Challenges and Errors Encountered:
Error with Table and Definition Mismatch:

While building the table schema in the DataFrame, I encountered errors related to mismatches in the length of the table names and their definitions. Initially, there was a ValueError that indicated all arrays must be of the same length. After correcting this, the DataFrame was successfully created.
Hallucinations or Inaccurate Table Mappings:

"Which movie genres have won the most Oscars?"
GPT-3.5 failed to provide accurate table suggestions when queried about genres. This indicated that the model was either hallucinating or the table structures were not well understood in relation to the data.
More Complex Queries:
When I tried more complex queries, such as combining different entities (e.g., movies and revenue), GPT-3.5 sometimes returned irrelevant or incomplete table mappings.
What Worked Well:
Simpler questions related directly to entities defined in the database schema worked well, and GPT-3.5 accurately provided the required tables in JSON format. Examples of this include questions related to specific actors, movies, or awards.
The model responded accurately when the queries were direct and tied closely to predefined table names and definitions.
What Didn’t Work Well:
For more abstract queries, such as asking about movie genres or attempting to combine too many fields in one query, GPT-3.5 struggled to provide accurate responses.
Additionally, when questions were vague or implied a deeper relationship between entities (like revenue and award wins), GPT-3.5 returned tables that didn’t directly match the expected SQL tables.
What Did I Learn:
Through this exercise, I learned the importance of structuring data and prompts carefully when working with language models. GPT-3.5 was highly effective at identifying simple and direct table mappings, but it struggled when the queries became more complex or ambiguous. The quality of the prompt directly influenced the quality of the response, and having clear table definitions helped in generating more accurate SQL queries. Furthermore, this lab demonstrated that while GPT-3.5 is powerful, it still has limitations in understanding intricate database relationships.

